# Test consistency between `legacy` and new `output` module

*Test if the two `carsus` interfaces make identical atomic files, **step by step**.*

> **IMPORTANT:** `pytest-notebook` runs the notebook and `diff`'s every cell against the previously saved result. So, the result of the test depends on the `diff` output and indirectly on the assertion. Logging is disabled to avoid comparing unnecesary strings. Keep in mind that the functions `assert_frame_equal` and `assert_series_equal` returns nothing when objects are identical.

---

In [1]:
import logging

logging.disable(logging.CRITICAL)

In [2]:
import pandas as pd
from pandas.testing import assert_frame_equal, assert_series_equal

In [3]:
GFALL_IONS = "H-Be"
CHIANTI_IONS = "H-He"
CHIANTI_DB_VERSION = "9.0.1"

## 1. Create `old` atomic data

In [4]:
from carsus import init_db
from carsus.io.nist import NISTIonizationEnergiesIngester, NISTWeightsCompIngester
from carsus.io.kurucz import GFALLIngester
from carsus.io.chianti_ import ChiantiIngester
from carsus.io.zeta import KnoxLongZetaIngester
from carsus.io.output import AtomData

session = init_db()

weightscomp_ingester = NISTWeightsCompIngester(session)
weightscomp_ingester.ingest()
session.commit()

ioniz_energies_ingester = NISTIonizationEnergiesIngester(session, spectra=GFALL_IONS)
ioniz_energies_ingester.ingest(ionization_energies=True, ground_levels=True)
session.commit()

gfall_ingester = GFALLIngester(session, ions=GFALL_IONS)
gfall_ingester.ingest(levels=True, lines=True)
session.commit()

chianti_ingester = ChiantiIngester(session, ions=CHIANTI_IONS)
chianti_ingester.ingest(levels=True, lines=True, collisions=True)
session.commit()

zeta_ingester = KnoxLongZetaIngester(session)
zeta_ingester.ingest()
session.commit()

old_atom_data = AtomData(session,
                         selected_atoms=GFALL_IONS, 
                         chianti_short_name=f"chianti_v{CHIANTI_DB_VERSION}", 
                         chianti_ions=CHIANTI_IONS)

 ChiantiPy version 0.8.4 
 found PyQt5 widgets
 using PyQt5 widgets


## 2. Create `new` atomic data

In [5]:
from carsus.io.nist import NISTWeightsComp, NISTIonizationEnergies
from carsus.io.kurucz import GFALLReader
from carsus.io.zeta import KnoxLongZeta
from carsus.io.chianti_ import ChiantiReader
from carsus.io.output import TARDISAtomData

atomic_weights = NISTWeightsComp()
ionization_energies = NISTIonizationEnergies(GFALL_IONS)
gfall_reader = GFALLReader(ions=GFALL_IONS)
chianti_reader = ChiantiReader(ions=CHIANTI_IONS, collisions=True, priority=20)
zeta_data = KnoxLongZeta()

new_atom_data = TARDISAtomData(atomic_weights,
                               ionization_energies,
                               gfall_reader,
                               zeta_data,
                               chianti_reader)

## 3. Create objects

In [6]:
# to do: make consistent with legacy `AtomData` class
old_atomic_weights = old_atom_data.atom_masses_prepared
old_atomic_weights['symbol'] = old_atomic_weights['symbol'].str.decode('UTF-8')
old_atomic_weights['name'] = old_atomic_weights['name'].str.decode('UTF-8')

new_atomic_weights = new_atom_data.atomic_weights.base.loc[1:4]  # H-Be

In [7]:
# to do: make consistent with legacy `AtomData` class
old_ionization_energies = old_atom_data.ionization_energies_prepared
new_ionization_energies = new_atom_data.ionization_energies_prepared

In [8]:
old_levels_all = old_atom_data._get_all_levels_data().reset_index()
new_levels_all = new_atom_data._get_all_levels_data().drop(columns=["ds_id"])

In [9]:
old_levels = old_atom_data.levels
new_levels = new_atom_data.levels.drop(columns=["ds_id"])

In [10]:
old_levels_prepared = old_atom_data.levels_prepared
new_levels_prepared = new_atom_data.levels_prepared

In [11]:
old_lines_all = old_atom_data._get_all_lines_data()
new_lines_all = new_atom_data._get_all_lines_data().drop(columns=["ds_id"])

In [12]:
old_lines = old_atom_data.lines
new_lines = new_atom_data.lines.drop(columns=["ds_id"])

In [13]:
old_lines_prepared = old_atom_data.lines_prepared
new_lines_prepared = new_atom_data.lines_prepared

In [14]:
old_macro_atom = old_atom_data.macro_atom
new_macro_atom = new_atom_data.macro_atom

In [15]:
old_macro_atom_prepared = old_atom_data.macro_atom_prepared
new_macro_atom_prepared = new_atom_data.macro_atom_prepared

In [16]:
old_macro_atom_references = old_atom_data.macro_atom_references
new_macro_atom_references = new_atom_data.macro_atom_references

In [17]:
old_macro_atom_references_prepared = old_atom_data.macro_atom_references_prepared
new_macro_atom_references_prepared = new_atom_data.macro_atom_references_prepared

In [18]:
old_collisions = old_atom_data.collisions.drop(columns=["btemp", "bscups"])
new_collisions = new_atom_data.collisions.drop(columns=["btemp", "bscups"])

In [19]:
old_collisions_prepared = old_atom_data.collisions_prepared
new_collisions_prepared = new_atom_data.collisions_prepared

In [20]:
# to do: make consistent with legacy `AtomData` class
old_zeta_data = old_atom_data.zeta_data
new_zeta_data = new_atom_data.zeta_data.base

## 4. Assert equal

In [21]:
assert_frame_equal(old_atomic_weights, new_atomic_weights)

In [22]:
assert_series_equal(old_ionization_energies, new_ionization_energies)

In [23]:
assert_frame_equal(old_levels_all, new_levels_all)

In [24]:
assert_frame_equal(old_levels, new_levels)

In [25]:
assert_frame_equal(old_levels_prepared, new_levels_prepared)

In [26]:
assert_frame_equal(old_lines_all, new_lines_all)

In [27]:
assert_frame_equal(old_lines, new_lines)

In [28]:
assert_frame_equal(old_lines_prepared, new_lines_prepared)

In [29]:
assert_frame_equal(old_macro_atom, new_macro_atom)

In [30]:
assert_frame_equal(old_macro_atom_prepared, new_macro_atom_prepared)

In [31]:
assert_frame_equal(old_macro_atom_references, new_macro_atom_references)

In [32]:
assert_frame_equal(old_macro_atom_references_prepared, new_macro_atom_references_prepared)

In [33]:
assert_frame_equal(old_collisions, new_collisions)

In [34]:
assert_frame_equal(old_collisions_prepared, new_collisions_prepared)

In [35]:
assert_frame_equal(old_zeta_data, new_zeta_data)